### Case Study

Computer Vision with CNN - Project 1:

The case study is from a dataset from Kaggle.

Link to the Kaggle project site:

https://www.kaggle.com/c/plant-seedlings-classification (Links to an external site.)Links to an external site.

The dataset has to be downloaded from the above Kaggle web site.

Can you differentiate a weed from a crop seedling?

The ability to do so effectively can mean better crop yields and better stewardship of the environment.

The Aarhus University Signal Processing group, in collaboration with University of Southern Denmark, has recently released a dataset containing images of approximately 960 unique plants belonging to 12 species at several growth stages.

### Connect with Google Drive

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Functions for image augmentation...

In [0]:
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]
  
# dictionary of the transformations we defined earlier
available_transformations = {
    'rotate': random_rotation,
    'noise': random_noise,
    'horizontal_flip': horizontal_flip
}

### Read the images and generate the train and test dataset

In [0]:
#Import necessary libraries to fetch the train and test data...
import os
from zipfile import ZipFile
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf

tf.set_random_seed(42)

In [0]:
#Change the working directory to make our file path access simple.

os.chdir('/content/drive/My Drive/Colab Notebooks/Computer Vision/plant-seedlings-classification/')

In [0]:
#Extract the train Zip file content

#with ZipFile('train.zip', 'r') as z:
#  z.extractall()

In [0]:
#Extract the test Zip file content

#with ZipFile('test.zip', 'r') as z:
#  z.extractall()

In [0]:
#Read the test images in to an array...

X_test=[]
X_test_name=[]
os.chdir('/content/drive/My Drive/Colab Notebooks/Computer Vision/plant-seedlings-classification/test')

for i in os.listdir():
  try:
    dummy = cv2.imread(i)
    dummy = cv2.resize(dummy,(128,128)) #resize to have all the images of same size
    X_test.append(dummy)
    X_test_name.append(i)
  except Exception as e:
    print(e)

In [0]:
#Now lets read train data...

os.chdir('/content/drive/My Drive/Colab Notebooks/Computer Vision/plant-seedlings-classification/train')

In [37]:
#List all the contents in the train directory...
ls

 Black-grass/        'Common wheat'/      'Scentless Mayweed'/
 Charlock/           'Fat Hen'/           'Shepherds Purse'/
 Cleavers/           'Loose Silky-bent'/  'Small-flowered Cranesbill'/
'Common Chickweed'/   Maize/              'Sugar beet'/


In [38]:
#Train images are classified and placed in folders. The folder name should be taken as target (y_train).
os.listdir()

['Shepherds Purse',
 'Loose Silky-bent',
 'Small-flowered Cranesbill',
 'Sugar beet',
 'Fat Hen',
 'Scentless Mayweed',
 'Maize',
 'Common wheat',
 'Common Chickweed',
 'Cleavers',
 'Charlock',
 'Black-grass']

In [39]:
#There are 12 distinct categories. Let see on how the train data is split among these categories...
for i in os.listdir():
  path, dirs, files = next(os.walk(i))
  print (i + " : " + str(len(files)))

Shepherds Purse : 755
Loose Silky-bent : 756
Small-flowered Cranesbill : 754
Sugar beet : 755
Fat Hen : 757
Scentless Mayweed : 757
Maize : 754
Common wheat : 754
Common Chickweed : 755
Cleavers : 757
Charlock : 754
Black-grass : 755


In [40]:
#The data is imbalanced among categories. Lets balance it by augmentation...

from skimage import io

#Lets generate new images such that each category have same number of images, to eradicate imbalance and bias.
num_files_desired = 754
num_files = 0

for i in os.listdir():
    if (os.path.isdir(i)):
            Images = []
            folder_path = "/content/drive/My Drive/Colab Notebooks/Computer Vision/plant-seedlings-classification/train/" + i
            for j in os.listdir(i):
                Images.append(folder_path + "/" + j)
            
            Img_cnt = len(Images)
            Cnt_diff = num_files_desired - Img_cnt
            num_generated_files = 0
            print ("In " + i + ", " + str(Cnt_diff) + " new images will be added by augmentation...")
            while num_generated_files < Cnt_diff:
                # random image from the folder
                image_path = random.choice(Images)
                # read image as an two dimensional array of pixels
                image_to_transform = sk.io.imread(image_path)

                # random num of transformation to apply
                num_transformations_to_apply = random.randint(1, len(available_transformations))

                num_transformations = 0
                transformed_image = None
                while num_transformations <= num_transformations_to_apply:
                    # random transformation to apply for a single image
                    key = random.choice(list(available_transformations))
                    transformed_image = available_transformations[key](image_to_transform)
                    num_transformations += 1

                    new_file_path = '%s/augmented_image_%s.png' % (folder_path, num_files)

                    # write image to the disk
                    io.imsave(new_file_path, transformed_image)
                    num_generated_files += 1
                    num_files += 1

In Shepherds Purse, -1 new images will be added by augmentation...
In Loose Silky-bent, -2 new images will be added by augmentation...
In Small-flowered Cranesbill, 0 new images will be added by augmentation...
In Sugar beet, -1 new images will be added by augmentation...
In Fat Hen, -3 new images will be added by augmentation...
In Scentless Mayweed, -3 new images will be added by augmentation...
In Maize, 0 new images will be added by augmentation...
In Common wheat, 0 new images will be added by augmentation...
In Common Chickweed, -1 new images will be added by augmentation...
In Cleavers, -3 new images will be added by augmentation...
In Charlock, 0 new images will be added by augmentation...
In Black-grass, -1 new images will be added by augmentation...


In [41]:
#Count of Images in each category after augmentation...
for i in os.listdir():
  path, dirs, files = next(os.walk(i))
  print (i + " : " + str(len(files)))

Shepherds Purse : 755
Loose Silky-bent : 756
Small-flowered Cranesbill : 754
Sugar beet : 755
Fat Hen : 757
Scentless Mayweed : 757
Maize : 754
Common wheat : 754
Common Chickweed : 755
Cleavers : 757
Charlock : 754
Black-grass : 755


In [42]:
X_train = []
y_train = []

for i in os.listdir():
    print(i)
    if (os.path.isdir(i)):
            for j in os.listdir(i):
                try:
                    dummy = cv2.imread('/content/drive/My Drive/Colab Notebooks/Computer Vision/plant-seedlings-classification/train/' + i + "/" + j)
                    dummy = cv2.resize(dummy,(128,128)) #resize to have all the images of same size
                    X_train.append(dummy)
                    y_train.append(i)
                except Exception as e:
                    print(e)

Shepherds Purse
Loose Silky-bent
Small-flowered Cranesbill
Sugar beet
Fat Hen
Scentless Mayweed
Maize
Common wheat
Common Chickweed
Cleavers
Charlock
Black-grass


In [43]:
print ("No. of images in X_train: ", len(X_train))
print ("No. of images in X_test: ", len(X_test))
print ("No. of values in y_train: ", len(y_train))

No. of images in X_train:  9063
No. of images in X_test:  794
No. of values in y_train:  9063


In [44]:
print ("Shape of an image in X_train: ", X_train[0].shape)
print ("Shape of an image in X_test: ", X_test[0].shape)

Shape of an image in X_train:  (128, 128, 3)
Shape of an image in X_test:  (128, 128, 3)


In [0]:
#Get lable encoding for y_train

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

In [46]:
print("Total Plant categories (Unique Target): ", len(np.unique(y_train)))

Total Plant categories (Unique Target):  12


In [0]:
y_train = np.array(y_train)
X_train = np.array(X_train)

In [0]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=12)

### Divide the data set into Train and validation data sets

In [49]:
from sklearn.model_selection import train_test_split

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2)
print ("No. of images in train dataset: ", len(X_train2))
print ("No. of images in Validation dataset: ", len(X_val))

No. of images in train dataset:  7250
No. of images in Validation dataset:  1813


In [50]:
print ("X_train2 Shape: ", X_train2.shape)
print ("X_val Shape: ", X_val.shape)
print("y_train2 Shape: ", y_train2.shape)
print("y_val Shape: ", y_val.shape)

X_train2 Shape:  (7250, 128, 128, 3)
X_val Shape:  (1813, 128, 128, 3)
y_train2 Shape:  (7250, 12)
y_val Shape:  (1813, 12)


### Initialize & build the model

In [0]:
#Import necessary libraries to build the model...
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import adam
from keras.optimizers import sgd
from keras.layers import LeakyReLU

In [0]:
model = Sequential()
model.add(BatchNormalization(input_shape = (128,128,3)))
model.add(Convolution2D(32, (3,3), input_shape = (128, 128, 3), kernel_initializer = 'he_normal')) 
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Convolution2D(filters=64, kernel_size=5, padding='same', kernel_initializer = 'he_normal'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Convolution2D(filters=128, kernel_size=4, padding='same', kernel_initializer = 'he_normal'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Convolution2D(filters=128, kernel_size=3, padding='same', kernel_initializer = 'he_normal'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Convolution2D(filters=128, kernel_size=2, padding='same', kernel_initializer = 'he_normal'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=128, kernel_initializer = 'he_normal'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(units = 64, kernel_initializer = 'he_normal'))
model.add(LeakyReLU(alpha=0.1))
#model.add(Dropout(0.3))
model.add(Dense(units = 32, kernel_initializer = 'he_normal'))
model.add(LeakyReLU(alpha=0.1))
#model.add(Dropout(0.3))
model.add(Dense(units = 12, activation = 'softmax'))

In [0]:
#optimizer = adam(lr=0.001)
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [54]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 63, 63, 64)        51264     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 63, 63, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 64)       

In [55]:
history = model.fit(X_train2,y_train2,
                    epochs=30, 
                    validation_data=(X_val,y_val),
                    verbose = 1,
                    initial_epoch=0)

Train on 7250 samples, validate on 1813 samples
Epoch 1/30
7250/7250 [==============================] - 376s 52ms/step - loss: 1.8917 - acc: 0.3455 - val_loss: 1.2871 - val_acc: 0.5383
Epoch 2/30
7250/7250 [==============================] - 375s 52ms/step - loss: 1.0322 - acc: 0.6408 - val_loss: 0.8132 - val_acc: 0.7148
Epoch 3/30
7250/7250 [==============================] - 374s 52ms/step - loss: 0.7159 - acc: 0.7461 - val_loss: 0.6594 - val_acc: 0.7827
Epoch 4/30
7250/7250 [==============================] - 370s 51ms/step - loss: 0.4982 - acc: 0.8279 - val_loss: 0.4814 - val_acc: 0.8307
Epoch 5/30
7250/7250 [==============================] - 375s 52ms/step - loss: 0.3826 - acc: 0.8634 - val_loss: 0.5634 - val_acc: 0.8075
Epoch 6/30
7250/7250 [==============================] - 371s 51ms/step - loss: 0.3057 - acc: 0.8886 - val_loss: 0.4664 - val_acc: 0.8478
Epoch 7/30
4416/7250 [=================>............] - ETA: 2:16 - loss: 0.2406 - acc: 0.9119

KeyboardInterrupt: ignored

In [0]:
#### interrupted as taking a lot of time

### Optimize the model

**Already implemented the best activation function and optimizer.**


**Lets optimize the batch size, epoc and weight initialization.**

In [56]:
# Tune Network Weight Initialization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model(init_mode='uniform'):
  model = Sequential()
  model.add(BatchNormalization(input_shape = (128,128,3)))
  model.add(Convolution2D(32, (3,3), input_shape = (128, 128, 3), kernel_initializer = init_mode)) 
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Convolution2D(filters=64, kernel_size=5, padding='same', kernel_initializer = init_mode))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Convolution2D(filters=128, kernel_size=4, padding='same', kernel_initializer = init_mode))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Convolution2D(filters=128, kernel_size=3, padding='same', kernel_initializer = init_mode))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Convolution2D(filters=128, kernel_size=2, padding='same', kernel_initializer = init_mode))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Flatten()) 

  # fully connected layer
  model.add(Dense(units=128, kernel_initializer = init_mode))
  model.add(LeakyReLU(alpha=0.1))
  model.add(Dense(units = 64, kernel_initializer = init_mode))
  model.add(LeakyReLU(alpha=0.1))
  #model.add(Dropout(0.3))
  model.add(Dense(units = 32, kernel_initializer = init_mode))
  model.add(LeakyReLU(alpha=0.1))
  #model.add(Dropout(0.3))
  model.add(Dense(units = 12, activation = 'softmax', kernel_initializer = init_mode)) 
  model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
  return model

# create model
modelOp = KerasClassifier(build_fn=create_model, epochs=10, verbose=0)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)

grid = GridSearchCV(estimator=modelOp, param_grid=param_grid, cv=2)
grid_result = grid.fit(X_train2, y_train2)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

KeyboardInterrupt: ignored

In [0]:
#### interrupted as taking a lot of time

In [57]:
# Tune Network Weight Initialization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model(init_mode='uniform'):
  model = Sequential()
  model.add(BatchNormalization(input_shape = (128,128,3)))
  model.add(Convolution2D(32, (3,3), input_shape = (128, 128, 3), kernel_initializer = 'lecun_uniform')) 
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Convolution2D(filters=64, kernel_size=5, padding='same', kernel_initializer = 'lecun_uniform'))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Convolution2D(filters=128, kernel_size=4, padding='same', kernel_initializer = 'lecun_uniform'))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Convolution2D(filters=128, kernel_size=3, padding='same', kernel_initializer = 'lecun_uniform'))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Convolution2D(filters=128, kernel_size=2, padding='same', kernel_initializer = 'lecun_uniform'))
  model.add(LeakyReLU(alpha=0.1))
  model.add(MaxPooling2D(pool_size=2))
  #model.add(Dropout(0.2))

  model.add(Flatten()) 

  # fully connected layer
  model.add(Dense(units=128, kernel_initializer = 'lecun_uniform'))
  model.add(LeakyReLU(alpha=0.1))
  model.add(Dense(units = 64, kernel_initializer = 'lecun_uniform'))
  model.add(LeakyReLU(alpha=0.1))
  #model.add(Dropout(0.3))
  model.add(Dense(units = 32, kernel_initializer = 'lecun_uniform'))
  model.add(LeakyReLU(alpha=0.1))
  #model.add(Dropout(0.3))
  model.add(Dense(units = 12, activation = 'softmax')) 
  model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
  return model

# create model
modelOp = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=modelOp, param_grid=param_grid, n_jobs=1, scoring="accuracy", cv=2)
grid_result = grid.fit(X_train2, np.argmax(y_train2, axis=1))

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

KeyboardInterrupt: ignored

In [0]:
#### interrupted as taking a lot of time

### Final Model


**Implementing the optimum hyperparameter based on above results...**

In [0]:
model = Sequential()
model.add(BatchNormalization(input_shape = (128,128,3)))
model.add(Convolution2D(32, (3,3), input_shape = (128, 128, 3), kernel_initializer = 'lecun_uniform')) 
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Convolution2D(filters=64, kernel_size=5, padding='same', kernel_initializer = 'lecun_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Convolution2D(filters=128, kernel_size=4, padding='same', kernel_initializer = 'lecun_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Convolution2D(filters=128, kernel_size=3, padding='same', kernel_initializer = 'lecun_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Convolution2D(filters=128, kernel_size=2, padding='same', kernel_initializer = 'lecun_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=128, kernel_initializer = 'lecun_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(units = 64, kernel_initializer = 'lecun_uniform'))
model.add(LeakyReLU(alpha=0.1))
#model.add(Dropout(0.3))
model.add(Dense(units = 32, kernel_initializer = 'lecun_uniform'))
model.add(LeakyReLU(alpha=0.1))
#model.add(Dropout(0.3))
model.add(Dense(units = 12, activation = 'softmax'))

In [0]:
#optimizer = adam(lr=0.001)
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [60]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_5 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 63, 63, 64)        51264     
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 63, 63, 64)        0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 31, 31, 64)       

In [61]:
history = model.fit(X_train2,y_train2,
                    epochs=2, #epochs=30  ## taking a lot of time during execution 
                    validation_data=(X_val,y_val),
                    verbose = 1,
                    initial_epoch=0, batch_size=60)

Train on 7250 samples, validate on 1813 samples
Epoch 1/2
7250/7250 [==============================] - 380s 52ms/step - loss: 1.8089 - acc: 0.3546 - val_loss: 1.2852 - val_acc: 0.5345
Epoch 2/2
7250/7250 [==============================] - 370s 51ms/step - loss: 0.9407 - acc: 0.6679 - val_loss: 0.6721 - val_acc: 0.7716


### Predict the accuracy for both train and validation data

In [0]:
predictions_train = model.predict(X_train2)
predictions_val = model.predict(X_val)

In [63]:
from sklearn import metrics
print ("Train Accuracy: ", metrics.accuracy_score(np.argmax(y_train2, axis=1), np.argmax(predictions_train, axis=1)))

Train Accuracy:  0.7844137931034483


In [64]:
from sklearn.metrics import classification_report

print ("Classification Report for train data")
print(classification_report(np.argmax(y_train2, axis=1), np.argmax(predictions_train, axis=1)))

Classification Report for train data
              precision    recall  f1-score   support

           0       0.66      0.54      0.59       601
           1       0.92      0.85      0.88       602
           2       0.88      0.87      0.88       595
           3       0.84      0.88      0.86       618
           4       0.82      0.82      0.82       622
           5       0.69      0.62      0.65       611
           6       0.66      0.69      0.68       603
           7       0.83      0.93      0.87       617
           8       0.65      0.80      0.71       602
           9       0.76      0.76      0.76       595
          10       0.94      0.88      0.91       594
          11       0.79      0.78      0.78       590

    accuracy                           0.78      7250
   macro avg       0.79      0.78      0.78      7250
weighted avg       0.79      0.78      0.78      7250



In [65]:
print ("Val Accuracy: ", metrics.accuracy_score(np.argmax(y_val, axis=1), np.argmax(predictions_val, axis=1)))

Val Accuracy:  0.7716492002206288


In [66]:
print ("Classification Report for Validation data")
print(classification_report(np.argmax(y_val, axis=1), np.argmax(predictions_val, axis=1)))

Classification Report for Validation data
              precision    recall  f1-score   support

           0       0.63      0.46      0.53       154
           1       0.89      0.84      0.86       152
           2       0.88      0.90      0.89       162
           3       0.78      0.89      0.83       137
           4       0.79      0.82      0.80       132
           5       0.69      0.63      0.66       146
           6       0.62      0.72      0.67       153
           7       0.80      0.90      0.85       137
           8       0.66      0.76      0.71       155
           9       0.78      0.74      0.76       160
          10       0.92      0.82      0.87       160
          11       0.82      0.80      0.81       165

    accuracy                           0.77      1813
   macro avg       0.77      0.77      0.77      1813
weighted avg       0.77      0.77      0.77      1813



**Observations:**

The category 0, 5 and 6 have very less f1 score. We should make the model learn more on these categories. We can do more augmentation in these categories and make model learn better.

### Predict Test

In [0]:
X_test = np.array(X_test)
predictions = model.predict(X_test)

In [0]:
FinalPred = np.argmax(predictions, axis=1)

In [0]:
y_test = le.inverse_transform(FinalPred)

In [0]:
X_test_name = np.array(X_test_name)
y_test = np.array(y_test)
dataset = pd.DataFrame({'file': X_test_name, 'species': y_test}, columns=['file', 'species'])

In [71]:
dataset.head()

,file,species
0,043449b0b.png,Sugar beet
1,04814f36d.png,Shepherds Purse
2,03ef36742.png,Scentless Mayweed
3,0437393b1.png,Fat Hen
4,03566743d.png,Shepherds Purse


In [0]:
dataset.to_csv("/content/drive/My Drive/Colab Notebooks/Computer Vision/plant-seedlings-classification/ResultsNipun.csv", index=False)